<a href="https://colab.research.google.com/github/Yuns-u/KoBERT_emotion_Classifier/blob/main/KoBERT_Emotion_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT를 위한 환경 설정

In [1]:
#KoBERT를 사용하기 위한 라이브러리 설정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.6 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595733 sha256=3df54e10b1fe0cabbf82ec80ba4381561f0b1395cb86ea2584b3f8ebd71884ce
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 769 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 3.0 MB 38.0 MB/s 


In [2]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3q3uxyhh
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3q3uxyhh
     |████████████████████████████████| 131 kB 5.3 MB/s 
     |████████████████████████████████| 3.6 MB 42.0 MB/s 
     |████████████████████████████████| 3.4 MB 31.5 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
     |████████████████████████████████| 61 kB 385 kB/s 
     |████████████████████████████████| 3.3 MB 37.8 MB/s 
     |████████████████████████████████| 79 kB 4.0 MB/s 
     |████████████████████████████████| 8.5 MB 36.0 MB/s 
     |████████████████████████████████| 127 kB 35.7 MB/s 
  Created wheel for kobert: filename=kobert-0.2.1-py3-none-any.whl size=15267 sha256=df2031677346f72a0892dc54a04ca557bf25aafed7a92c7fe94380d36f7619c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-516jftj4/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [3]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#GPU 사용
device = torch.device("cuda:0")

In [6]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# AI Hub에서 받아온 데이터 전처리하기

- 공포 : 5468 
- 놀람 : 5898 
- 분노 : 5665
- 슬픔 : 5267
- 중립 : 4830
- 행복 : 6037
- 혐오 : 5429

총 38594개의 말뭉치이다.

위처럼 일곱가지의 감정으로 분류되어 있지만 각 감정의 데이터 수 차이가 있다.
또한 감정의 수가 7가지나 되기 때문에 목적에 맞게 긍정(1), 부정(0), 중립(0.5) 정도로 분류하면 1:다 대화에서의 감정을 파악하는 데에 도움이 될 것이라고 생각한다.

In [7]:
import pandas as pd

In [8]:
from google.colab import files
myfile = files.upload()

Saving korean_oneoff_chats.csv to korean_oneoff_chats.csv


In [9]:
#pd.read_csv로 csv파일 불러오기
df = pd.read_csv('korean_oneoff_chats.csv')
df

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [10]:
#결측치 확인
df.isna().sum()

Sentence    0
Emotion     0
dtype: int64

## Emotion별 특징 분석하기
행복은 긍정이고 혐오는 부정인 것은 직관적으로 알 수 있지만 놀람, 공포 등 긍정인지 부정인지 명확하게 구분하기 어려운 감정말뭉치들이 있는 것으로 보인다.

해당 말뭉치가 단발성이라는 점에서 반어법 등의 유무를 살펴볼 수는 없었다. 
- 긍정: 행복
- 중립: 중립
- 부정: 혐오

공포, 놀람, 분노, 슬픔은 어떻게 나눌 수 있는가?

In [11]:
df['Emotion'].unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

일단 자의로 감정을 분석하는 것은 오히려 모델의 성능을 해칠 수 있으므로 7가지 감정에 대해 분류를 해보도록 하겠다.

- 혐오 : 0
- 분노 : 1
- 공포 : 2
- 중립 : 3
- 슬픔 : 4
- 놀람 : 5 
- 행복 : 6


In [12]:
df.loc[(df['Emotion']=='혐오'), 'Emotion'] = 0 #혐오를 0으로 표현
df.loc[(df['Emotion']=='분노'), 'Emotion'] = 1 #분노를 1로 표현
df.loc[(df['Emotion']=='공포'), 'Emotion'] = 2 #공포를 2로 표현
df.loc[(df['Emotion']=='중립'), 'Emotion'] = 3 #중립을 3으로 표현
df.loc[(df['Emotion']=='슬픔'), 'Emotion'] = 4 #슬픔을 4로 표현
df.loc[(df['Emotion']=='놀람'), 'Emotion'] = 5 #놀람을 5로 표현
df.loc[(df['Emotion']=='행복'), 'Emotion'] = 6 #행복을 6으로 표현

In [13]:
df.sample(10)

,Sentence,Emotion
10380,그 동안 도대체 무엇을 보아 온 것일까.,5
3143,내가 급하게 다가가봤자서툴러서 쉬운여자로만 보일거같고,2
7135,저 나라도 멕시코 수준인가보네,5
4327,재석이 오빠가 연예대상 받아야 하는대..,2
27924,역시 명불허전 똥싸개 라모스 ㅋㅋㅋ,6
28629,이은미씨 애국가 울컥하네요,6
7999,미쳐도 단단히 미쳤어~~~,5
18848,발그림 일단 미안 ..,4
8849,별 기능이 다있더군요.,5
15438,김대중 노무현 이 두 놈 때문에 c8,1


In [14]:
df.describe()

,Sentence,Emotion
count,38594,38594
unique,38509,7
top,감사합니다,6
freq,4,6037


In [15]:
#각 행의 데이터를 리스트로 묶어주고 하나의 리스트에 넣기
data_list = []

for q, label in zip(df['Sentence'], df['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [16]:
print(data_list[0])

['언니 동생으로 부르는게 맞는 일인가요..??', '2']


# 데이터셋 나누기

In [17]:
from sklearn.model_selection import train_test_split
                                                         
train_li, test_li = train_test_split(data_list, test_size=0.2, random_state=0)

In [18]:
print(len(train_li))
print(len(test_li))

30875
7719


# KoBERT 입력 데이터로 데이터셋 만들기

In [19]:
#BERT 모델에 들어가기 위한 데이터셋을 만들어주는 사용자 클래스 만들기
class BERTdata(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return(len(self.labels))

In [20]:
# KoBERT의 파라미터 세팅
max_len = 64             #텍스트 데이터 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5           #반복학습 수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

# 토큰화

In [21]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [22]:
#위의 클래스를 통해 데이터프레임에 있던 데이터를 토큰화해주기
train_data = BERTdata(train_li, 0, 1, tok, max_len, True, False)
test_data = BERTdata(test_li, 0, 1, tok, max_len, True, False)

In [23]:
train_data[0]

(array([   2, 1171, 7095,  835, 6079, 1409, 5400, 5886, 5655,  940,  633,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(12, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [24]:
#torch형식의 데이터셋으로 만들어주어 koBERT에 학습시킬 수 있는 형태로 만들어주기

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# 감정 분류 KoBERT 학습 모델 만들기

In [25]:
class KoBERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##분류할 클래스의 수에 맞춰 조정
                 dr_rate=None,
                 params=None):
        super(KoBERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
model = KoBERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [29]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [30]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [31]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
train_dataloader

# 모델 학습시키기

In [34]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/483 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.9311565160751343 train acc 0.203125
epoch 1 batch id 201 loss 1.4142661094665527 train acc 0.28544776119402987
epoch 1 batch id 401 loss 1.2860273122787476 train acc 0.38209164588528677
epoch 1 train acc 0.4038620983820259


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.5513250953591862


  0%|          | 0/483 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.6282719373703003 train acc 0.84375
epoch 5 batch id 201 loss 0.5231648683547974 train acc 0.7981187810945274
epoch 5 batch id 401 loss 0.440483957529068 train acc 0.8091879675810474
epoch 5 train acc 0.8112143911509854


  0%|          | 0/121 [00:00<?, ?it/s]

epoch 5 test acc 0.5534375662216571


batch size 64, epoch 5로 학습한 결과 1시간 정도가 소요되었으며 마지막 정확도는 0.5534375662216571로 출력되었다.

# 새로운 문장들로 테스트하기

위에서 훈련시킨 모델에 새로운 문장을 넣어 감정 분류를 잘 하는지 테스트해볼 수 있다.

In [35]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [40]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTdata(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("혐오")
            elif np.argmax(logits) == 1:
                test_eval.append("분노")
            elif np.argmax(logits) == 2:
                test_eval.append("공포")
            elif np.argmax(logits) == 3:
                test_eval.append("중립")
            elif np.argmax(logits) == 4:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 5:
                test_eval.append("놀람")
            elif np.argmax(logits) == 6:
                test_eval.append("행복")

        print(">> 예측된 감정: " + test_eval[0])

In [41]:
#문장을 입력하면 감정을 예측해주는 입력기
#0 입력시 종료
end = 1
while end == 1 :
    sentence = input("입력된 문장: ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

입력된 문장: 피드백 설문했는데 사진이 안보내져서 인증할수가 없구만!? 나도 깊티콘 받을래~~~~~~!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 예측된 감정: 행복


입력된 문장: 믿음이 가네요 🙃
>> 예측된 감정: 행복


입력된 문장: 애플워치가 필요한 순간이군?
>> 예측된 감정: 중립


입력된 문장: 셀카 유출인가?
>> 예측된 감정: 놀람


입력된 문장: 아쉽다! 기프티콘 주고싶은데 사진 전송이 안되는군아 너무 아쉽네 ㅠㅠ
>> 예측된 감정: 슬픔


입력된 문장: 오 연예인 같다 ㅋㅋㅋㅋㅋㅋㅋ 화보집인줄
>> 예측된 감정: 놀람


입력된 문장: 汚いです
>> 예측된 감정: 놀람


입력된 문장: 🤔🤔🤔
>> 예측된 감정: 놀람


입력된 문장: 오오 배그 재밌죠 2부 봐야지
>> 예측된 감정: 행복


입력된 문장: 그럼 ㅇㅈ
>> 예측된 감정: 놀람


입력된 문장: 사진 거꾸로 보면 진짜 토끼가 보이네 ㄷㄷ
>> 예측된 감정: 놀람


입력된 문장: 얼 플레 복구 ㅊㅋㅊㅋ
>> 예측된 감정: 공포


입력된 문장: 1인상이랬는데... 이건 4인상 아닌가요
>> 예측된 감정: 공포


입력된 문장: 먼던건좀...
>> 예측된 감정: 공포




KeyboardInterrupt: ignored

# 추가사항

- 1:다 채팅의 경우 이모티콘도 감정 표현이 될 것 같은데... 하트 등의 이모티콘을 행복으로 표현한다면 좀 더 정확한 측정이 가능하지 않을까?
- 유행어나 맥락 상 반의어나 별 의미가 없는 경우에도 공포나 놀람 등의 감정으로 분류되는 경우도 있음
- 일본어나 영어 등을 사용할 경우도 있는 듯함
- 약어에 대한 전처리가 필요한 것으로 보임